# PIPELINE

A medida que hemos ido aprendiendo más y más funcionalidades de scikit, vemos que la cantidad de operaciones a realizar sobre nuestros datos aumenta. Por ejemplo, podemos tener que hacer varias operaciones de preprocessing, una feature selection, ajuste de los hiperparámetros del algoritmo y todo ello además, encerrado en un cross-validation. Para hacer todas estas operaciones de manera automática, scikit incorpora un módulo llamado `Pipeline`, que permite colocar diferentes operaciones que van a actuar de manera secuencial sobre los datos. 

Las ventajas de usar este módulo son:

- **Convenciencia y encapsulacion**: Sólo hay que llamar una vez a `fit` y `predict` para que se realice sobre toda la secuencia de pasos
- **Selección de parámetros unificada**: Cada paso seguramente dependa de algún hiperparámetro que ajustar. Se pueden usar los metodos *grid search* para explorar dichos parámetros a la vez
- **Seguridad**: Evita resultados demasiado optimistas al hacer algún paso antes de cross-validation. Una vez definido el pipeline, si lo metemos dentro de un esquema de cross-validation, **todos** los pasos se realizan sobre el training set. 

Todas las operaciones menos la última, transforman la data. Por tanto, todos los algoritmos que implementen estos pasos han de tener un método `fit` y otro `transform`. El último puede ser de cualquier tipo, aunque lo normal es que sea el clasificador que usamos para predecir.

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [ ]:
from sklearn.datasets import load_iris

In [ ]:
iris = load_iris()

In [ ]:
X = iris['data']
y = iris['target']

Voy a importar las tres operaciones que voy a hacer, que van a consistir en seleccionar la mejor feature mediante un anova, un reescalado de los datos,  y por último, realizar la clasificación mediante un algoritmo de tipo Logistic Regression

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression

scaler = StandardScaler()
feat_sel = SelectKBest(k=1)
clf = LogisticRegression()

Podemos encadenar fácilmente los tres pasos anteriores mediante la clase `Pipeline`

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
Pipeline?

In [ ]:
pip = Pipeline([
    ('feat_sel', feat_sel), # Paso 1: hacer una feature selection usando ANOVA
    ('normalizer', StandardScaler()), # Paso 2: Estandarizar los datos
    ('clf', LogisticRegression()) # Paso 3- Clasificador
])

In [ ]:
#Podemos comprobar los pasos
pip.steps

In [ ]:
from sklearn.model_selection import cross_val_score

print(np.mean(cross_val_score(pip, X, y, cv=10)))

Cada paso de este Pipeline puede tener algún parámetro que se pueda querer optimizar, ya que puede tener relevancia en el rendimiento final. Para ello, podemos usar un GridSearch, especificando el parámetro dentro del paso del Pipeline que estamos variando. Sólo tienes que escribir el nombre del paso en el Pipeline (el primer elemento de cada tupla), seguido por "__" y el parámetro a variar. 

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'feat_sel__k': [1,2,3,4], 'clf__C': [0.01,0.1,1,10,100]}
grid = GridSearchCV(pip, param_grid=param_grid, cv = 3)

In [ ]:
grid.fit(X,y)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

Podemos acceder también a cada paso individual una vez ajustado todo el Pipeline

In [ ]:
print(grid.best_estimator_.named_steps['feat_sel'].get_support())

Si queremos ver ahora cómo generaliza esto, lo podemos hacer haciéndo un holdout antes del grid search o metiéndolo dentro de un cross-validation 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

grid.fit(X_train, y_train)
print(grid.score(X_test, y_test))

O usando cross-validation

In [ ]:
print(np.mean(cross_val_score(grid, X,y, cv = 10)))

Referencias: http://scikit-learn.org/stable/modules/pipeline.html#pipeline